In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../../data/FM_data/'
group_dir_ = '../../../data/FM_data/groups/random'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_50.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

218

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,2,2.154104,1
1,0,14,2.175915,1
2,0,15,2.244578,1
3,0,20,2.275416,1
4,0,21,2.088449,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)

In [6]:
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
9147,9147,2.064220
55,55,2.064220
6190,6190,2.018349
555,555,1.972477
1388,1388,1.972477
3825,3825,1.949541
1360,1360,1.949541
48,48,1.949541
355,355,1.949541
5935,5935,1.880734


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [12]:
p_list = []
r_list = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1

                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
            

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)

        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
#     print(avg_precisions)
#     print(avg_recalls)
    p_list.append(avg_precisions)
    r_list.append(avg_recalls)

In [13]:
for i in p_list:
    print(i)

for i in r_list:
    print(i)

[0.08142201834862385, 0.0798611111111111, 0.08217592592592593, 0.08255813953488372, 0.08217592592592593, 0.08064516129032258, 0.08101851851851852, 0.0798611111111111, 0.07738095238095238, 0.07875]
[0.1341743119266055, 0.11342592592592593, 0.1111111111111111, 0.1, 0.10069444444444445, 0.08986175115207373, 0.09722222222222222, 0.09259259259259259, 0.09285714285714286, 0.07875]
[0.17889908256880735, 0.15393518518518517, 0.13541666666666666, 0.13488372093023257, 0.11805555555555555, 0.11405529953917051, 0.11226851851851852, 0.1111111111111111, 0.10238095238095238, 0.08375]
[0.18577981651376146, 0.1724537037037037, 0.16666666666666666, 0.15232558139534882, 0.14814814814814814, 0.14400921658986174, 0.13425925925925927, 0.13194444444444445, 0.12380952380952381, 0.095]
[0.18922018348623854, 0.1840277777777778, 0.18055555555555555, 0.16511627906976745, 0.16898148148148148, 0.1497695852534562, 0.14583333333333334, 0.14351851851851852, 0.13452380952380952, 0.10625]
[0.1915137614678899, 0.1875, 0.

In [14]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [15]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)  
                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    print(avg_nDCGs)
#     print('----------------------------------------') 

[0.029788248672997543, 0.02974370114951184, 0.03006406579034011, 0.030203898654481226, 0.030064065790340114, 0.02982486412491986, 0.02825675074408038, 0.02891826152025805, 0.027605448775948612, 0.02970611151596492]
[0.056230418746284636, 0.04549985689774922, 0.04377933694921529, 0.03819245760950999, 0.03717628405027557, 0.03521572291581658, 0.03859813907326651, 0.038213572641752305, 0.03211331947916937, 0.028327851963411095]
[0.0829749382938107, 0.06682683617896096, 0.061556365932567386, 0.05822018410034251, 0.05023264311932121, 0.042724809831259515, 0.04839917665070716, 0.04569584465155406, 0.03442569224972508, 0.03213773116244926]
[0.08515893348821257, 0.0797401142427753, 0.07415011439460564, 0.06712251483802643, 0.06672478742508886, 0.055212240110309035, 0.059020409691273285, 0.05627422951337218, 0.04733197048073421, 0.03295821956796204]
[0.08575406565258653, 0.08664846329878052, 0.07996582582921224, 0.07444634639735517, 0.07454015917510334, 0.06284456509193748, 0.06458542661943675,